# 第5章: 大規模言語モデル

この章では, 大規模言語モデル (LLM; Large Language Model) の利用し, 様々なタスクに取り組む. 大規模言語モデルをプログラムからAPI経由で呼び出すことを想定しており, そのAPIの利用で費用が発生する可能性があることに留意せよ.

```{warning}
本章は, `code-cell` ではなく, Markdown のコードブロック内にコードを記述しているため, Google Colab上で直接実行できません.
```

In [1]:
# !pip install vllm

## 40. Zero-Shot推論

以下の問題の解答を作成せよ. ただし, 解答生成はzero-shot推論とせよ.

```
9世紀に活躍した人物に関係するできごとについて述べた次のア～ウを年代の古い順に正しく並べよ。

ア　藤原時平は，策謀を用いて菅原道真を政界から追放した。
イ　嵯峨天皇は，藤原冬嗣らを蔵人頭に任命した。
ウ　藤原良房は，承和の変後，藤原氏の中での北家の優位を確立した。
```

出典: [令和5年度第1回高等学校卒業程度認定試験問題](https://www.mext.go.jp/a_menu/koutou/shiken/kakomon/1411255_00010.htm) [日本史AB](https://www.mext.go.jp/content/20240523-mxt_syogai02-mext_000031286_03nihonshi.pdf) 問題 日本史B 1 問3

```python
import os
import google.generativeai as genai


genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

model = genai.GenerativeModel(
    model_name="gemini-2.0-flash",
    generation_config={
        "temperature": 0.6,
        "top_p": 0.9,
        "max_output_tokens": 512,
    }
)

prompt = """
あなたは日本の歴史学者です。次の問題に解答してください。

```
9世紀に活躍した人物に関係するできごとについて述べた次のア～ウを年代の古い順に正しく並べよ。

ア　藤原時平は，策謀を用いて菅原道真を政界から追放した。
イ　嵯峨天皇は，藤原冬嗣らを蔵人頭に任命した。
ウ　藤原良房は，承和の変後，藤原氏の中での北家の優位を確立した。

解答：
```
"""

response = model.generate_content(prompt).text

print(response)
```

```bash
はい、承知いたしました。9世紀に活躍した人物に関連する出来事を年代の古い順に並べます。

*   **イ　嵯峨天皇は，藤原冬嗣らを蔵人頭に任命した。**

    嵯峨天皇は809年から823年まで在位し、藤原冬嗣を蔵人頭に任命したのは弘仁元年（810年）のことです。これにより、天皇の側近として政治に関与する蔵人という役職が重要視されるようになり、藤原氏が政治の中枢に進出する足
がかりとなりました。

*   **ウ　藤原良房は，承和の変後，藤原氏の中での北家の優位を確立した。**

    承和の変は承和9年（842年）に起こりました。藤原良房はこの変を鎮圧し、皇太子擁立に成功することで、藤原北家の地位を確立しました。

*   **ア　藤原時平は，策謀を用いて菅原道真を政界から追放した。**

    藤原時平が菅原道真を大宰府に左遷したのは昌泰4年（901年）のことです。これにより、藤原氏は政治の実権を完全に掌握し、摂関政治の基礎を築きました。

したがって、年代の古い順に並べると、**イ → ウ → ア** となります。

**解答：イ → ウ → ア**
```

## 41. Few-Shot推論

以下の問題と解答を与え, 問題40で示した質問の解答をfew-shot推論（この場合は4-shot推論）で生成せよ.

```
日本の近代化に関連するできごとについて述べた次のア～ウを年代の古い順に正しく並べよ。

ア　府知事・県令からなる地方官会議が設置された。
イ　廃藩置県が実施され，中央から府知事・県令が派遣される体制になった。
ウ　すべての藩主が，天皇に領地と領民を返還した。

解答: ウ→イ→ア
```

出典: [令和5年度第1回高等学校卒業程度認定試験問題](https://www.mext.go.jp/a_menu/koutou/shiken/kakomon/1411255_00010.htm) [日本史AB 問題](https://www.mext.go.jp/content/20240523-mxt_syogai02-mext_000031286_03nihonshi.pdf) 日本史A 1 問8

```
江戸幕府の北方での対外的な緊張について述べた次の文ア～ウを年代の古い順に正しく並べよ。

ア　レザノフが長崎に来航したが，幕府が冷淡な対応をしたため，ロシア船が樺太や択捉島を攻撃した。
イ　ゴローウニンが国後島に上陸し，幕府の役人に捕らえられ抑留された。
ウ　ラクスマンが根室に来航し，漂流民を届けるとともに通商を求めた。

解答: ウ→ア→イ
```

出典: [令和5年度第1回高等学校卒業程度認定試験問題](https://www.mext.go.jp/a_menu/koutou/shiken/kakomon/1411255_00010.htm) [日本史AB 問題](https://www.mext.go.jp/content/20240523-mxt_syogai02-mext_000031286_03nihonshi.pdf) 日本史B 3 問3

```
中居屋重兵衛の生涯の期間におこったできごとについて述べた次のア～ウを，年代の古い順に正しく並べよ。

ア　アヘン戦争がおこり，清がイギリスに敗北した。
イ　異国船打払令が出され，外国船を撃退することが命じられた。
ウ　桜田門外の変がおこり，大老の井伊直弼が暗殺された。

解答: イ→ア→ウ
```

出典: [令和4年度第1回高等学校卒業程度認定試験問題](https://www.mext.go.jp/a_menu/koutou/shiken/kakomon/1411255_00007.htm) [日本史 問題](https://www.mext.go.jp/content/20240513-mxt_syogai02-mext_00002452_03nihonshi.pdf) 日本史A 1 問1

```
加藤高明が外務大臣として提言を行ってから、内閣総理大臣となり演説を行うまでの時期のできごとについて述べた次のア～ウを，年代の古い順に正しく並べよ。

ア　朝鮮半島において，独立を求める大衆運動である三・一独立運動が展開された。
イ　関東大震災後の混乱のなかで，朝鮮人や中国人に対する殺傷事件がおきた。
ウ　日本政府が，袁世凱政府に対して二十一カ条の要求を突き付けた。

解答: ウ→ア→イ
```

出典: [令和4年度第1回高等学校卒業程度認定試験問題](https://www.mext.go.jp/a_menu/koutou/shiken/kakomon/1411255_00007.htm) [日本史 問題](https://www.mext.go.jp/content/20240513-mxt_syogai02-mext_00002452_03nihonshi.pdf) 日本史A 2 問4

```python
import os
import google.generativeai as genai


genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

model = genai.GenerativeModel(
    model_name="gemini-2.0-flash",
    generation_config={
        "temperature": 0.6,
        "top_p": 0.9,
        "max_output_tokens": 512,
    }
)

prompt = """
あなたは日本の歴史学者です。以下に問題とその解答の例を数件示します。

```
日本の近代化に関連するできごとについて述べた次のア～ウを年代の古い順に正しく並べよ。

ア　府知事・県令からなる地方官会議が設置された。
イ　廃藩置県が実施され，中央から府知事・県令が派遣される体制になった。
ウ　すべての藩主が，天皇に領地と領民を返還した。

解答：ウ→イ→ア
```

```
江戸幕府の北方での対外的な緊張について述べた次の文ア～ウを年代の古い順に正しく並べよ。

ア　レザノフが長崎に来航したが，幕府が冷淡な対応をしたため，ロシア船が樺太や択捉島を攻撃した。
イ　ゴローウニンが国後島に上陸し，幕府の役人に捕らえられ抑留された。
ウ　ラクスマンが根室に来航し，漂流民を届けるとともに通商を求めた。

解答：ウ→ア→イ
```

```
中居屋重兵衛の生涯の期間におこったできごとについて述べた次のア～ウを，年代の古い順に正しく並べよ。

ア　アヘン戦争がおこり，清がイギリスに敗北した。
イ　異国船打払令が出され，外国船を撃退することが命じられた。
ウ　桜田門外の変がおこり，大老の井伊直弼が暗殺された。

解答：イ→ア→ウ
```

```
加藤高明が外務大臣として提言を行ってから、内閣総理大臣となり演説を行うまでの時期のできごとについて述べた次のア～ウを，年代の古い順に正しく並べよ。

ア　朝鮮半島において，独立を求める大衆運動である三・一独立運動が展開された。
イ　関東大震災後の混乱のなかで，朝鮮人や中国人に対する殺傷事件がおきた。
ウ　日本政府が，袁世凱政府に対して二十一カ条の要求を突き付けた。

解答：ウ→ア→イ
```

上記の例をもとに、以下の問題に解答してください。

```
9世紀に活躍した人物に関係するできごとについて述べた次のア～ウを年代の古い順に正しく並べよ。

ア　藤原時平は，策謀を用いて菅原道真を政界から追放した。
イ　嵯峨天皇は，藤原冬嗣らを蔵人頭に任命した。
ウ　藤原良房は，承和の変後，藤原氏の中での北家の優位を確立した。

解答：
```
"""

response = model.generate_content(prompt).text

print(response)
```

```bash
解答：イ→ウ→ア
```

## 42. 多肢選択問題の正解率

[JMMLU](https://github.com/nlp-waseda/JMMLU) のいずれかの科目を大規模言語モデルに解答させ, その正解率を求めよ.

In [2]:
# !git clone https://github.com/nlp-waseda/JMMLU.git

```python
import os
import time
from tqdm import tqdm
import pandas as pd
import google.generativeai as genai

interval = 4.1

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

model = genai.GenerativeModel(
    model_name="gemini-2.0-flash",
    generation_config={
        "temperature": 0.6,
        "top_p": 0.9,
        "max_output_tokens": 512,
    }
)

df = pd.read_csv("JMMLU/JMMLU/abstract_algebra.csv", header=None)
df.columns = ["question", "selA", "selB", "selC", "selD", "answer"]

valid_answers = ['A', 'B', 'C', 'D']
predictions = []
invalid_cnt = 0

system_prompt = "あなたは優れた抽象代数学の専門家です。以下の質問に対して所与の選択肢から正しい答えを選んでください。解答として許容される文字は「A」「B」「C」「D」のみです。解答は選択肢のいずれか1つを選ぶ形で行ってください。\n\n"

for _, row in tqdm(df.iterrows(), desc="Progress"):
    question_text =f"""
        {system_prompt}
        次の抽象代数学の問題を解いてください。
        
        row["question"]

        選択肢:
        A. {row['selA']}
        B. {row['selB']}
        C. {row['selC']}
        D. {row['selD']}
        正しい選択肢をA〜Dの中から1つ選び、解答はA〜Dの1文字のみで答えてください。\n\n"""

    response = model.generate_content(question_text)
    answer = response.text.strip().upper()
    time.sleep(interval)

    if answer not in valid_answers:
        clarification = f"""
            {question_text}
            あなたの先ほどの解答「{answer}」はA〜Dのいずれでもありません。
            この問題における正しい選択肢を、A〜Dの1文字のみで答えてください。"""
        clarified = model.generate_content(clarification).text.strip().upper()
        time.sleep(interval)
        if clarified in valid_answers:
            answer = clarified
        else:
            answer = "Invalid"
            invalid_cnt += 1

    predictions.append(answer)

valid_indices = [i for i, p in enumerate(predictions) if p in valid_answers]
correct_predictions = [predictions[i] == df.loc[i, 'answer'] for i in valid_indices]
accuracy = sum(correct_predictions) / len(correct_predictions) if correct_predictions else 0.0

print(f"\nAccuracy: {accuracy * 100:.2f}% (Number of invalid predictions: {invalid_cnt})")
```

```bash
Accuracy: 59.60% (Number of invalid predictions: 0)
```

## 43. 応答のバイアス

問題42において, 実験設定を変化させると正解率が変化するかどうかを調べよ. 実験設定の例としては, 大規模言語モデルの温度パラメータ, プロンプト, 多肢選択肢の順番, 多肢選択肢の記号などが考えられる.

```python
import os
import time
from tqdm import tqdm
import pandas as pd
import google.generativeai as genai

interval = 4.1

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

model = genai.GenerativeModel(
    model_name="gemini-2.0-flash",
    generation_config={
        "temperature": 0.6,
        "top_p": 0.9,
        "max_output_tokens": 512,
    }
)

df = pd.read_csv("JMMLU/JMMLU/abstract_algebra.csv", header=None)
df.columns = ["question", "selA", "selB", "selC", "selD", "answer"]

valid_answers = ['A', 'B', 'C', 'D']
predictions = []
invalid_cnt = 0

system_text = "あなたは抽象代数学の第一人者です。問題に対して最も適切な解答をA、B、C、Dのいずれか1文字のみで答えてください。途中の説明や理由付けは不要です。\n\n"

for _, row in tqdm(df.iterrows(), desc="Progress"):
    question_text =f"""
        {system_text}
        次の抽象代数学の問題を解いてください。
        
        【問題】
        {row['question']}
        
        【選択肢】
        A. {row['selA']}
        B. {row['selB']}
        C. {row['selC']}
        D. {row['selD']}
        
        【指示】
        最終的な答えをA、B、C、Dのいずれか1文字のみで示してください。その他の文章は不要です。"""

    response = model.generate_content(question_text)
    answer = response.text.strip().upper()
    time.sleep(interval)

    if answer not in valid_answers:
        clarification = f"""
            {question_text}
            あなたの先ほどの解答「{answer}」はA〜Dのいずれでもありません。
            この問題における正しい選択肢を、A〜Dの1文字のみで答えてください。"""
        clarified = model.generate_content(clarification).text.strip().upper()
        time.sleep(interval)
        if clarified in valid_answers:
            answer = clarified
        else:
            answer = "Invalid"
            invalid_cnt += 1

    predictions.append(answer)

valid_indices = [i for i, p in enumerate(predictions) if p in valid_answers]
correct_predictions = [predictions[i] == df.loc[i, 'answer'] for i in valid_indices]
accuracy = sum(correct_predictions) / len(correct_predictions) if correct_predictions else 0.0

print(f"\nAccuracy: {accuracy * 100:.2f}% (Number of invalid predictions: {invalid_cnt})")
```

```bash
Accuracy: 63.64% (Number of invalid predictions: 0)
```

## 44. 対話

以下の問いかけに対する応答を生成せよ.

> つばめちゃんは渋谷駅から東急東横線に乗り、自由が丘駅で乗り換えました。東急大井町線の大井町方面の電車に乗り換えたとき、各駅停車に乗車すべきところ、間違えて急行に乗車してしまったことに気付きました。自由が丘の次の急行停車駅で降車し、反対方向の電車で一駅戻った駅がつばめちゃんの目的地でした。目的地の駅の名前を答えてください。

参考: [東急線・みなとみらい線路線案内](https://www.tokyu.co.jp/railway/station/map.html)

```python
import os
import httpx
import base64
import google.generativeai as genai

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

model = genai.GenerativeModel(
    model_name="gemini-2.5-pro-exp-03-25",
    generation_config={
        "temperature": 0.6,
        "top_p": 0.9
    }
)

image_path = "https://www.tokyu.co.jp/railway/station/img/map_img01.jpg"
image = httpx.get(image_path)

system = "あなたは日本の鉄道員です。添付画像を参考にして、以下の問いに答えてください。\n"
user = "\n【問題】\nつばめちゃんは渋谷駅から東急東横線に乗り、自由が丘駅で乗り換えました。東急大井町線の大井町方面の電車に乗り換えたとき、各駅停車に乗車すべきところ、間違えて急行に乗車してしまったことに気付きました。自由が丘の次の急行停車駅で降車し、反対方向の電車で一駅戻った駅がつばめちゃんの目的地でした。目的地の駅の名前を答えてください。\n"

prompt = system + user
response = model.generate_content([{'mime_type':'image/jpeg', 'data': base64.b64encode(image.content).decode('utf-8')}, prompt]).text

print(f"System:\n\n{system}\nUser:\n{user}\nGemini:\n\n{response}")
```

```bash
System:

あなたは日本の鉄道員です。添付画像を参考にして、以下の問いに答えてください。

User:

【問題】
つばめちゃんは渋谷駅から東急東横線に乗り、自由が丘駅で乗り換えました。東急大井町線の大井町方面の電車に乗り換えたとき、各駅停車に乗車すべきところ、間違えて急行に乗車してしまったことに気付きました。自由が丘の次の
急行停車駅で降車し、反対方向の電車で一駅戻った駅がつばめちゃんの目的地でした。目的地の駅の名前を答えてください。

Gemini:

承知いたしました。鉄道員としてお答えします。

つばめちゃんが自由が丘駅 (OM10) から大井町方面の急行に乗車し、次に停車した駅は**大岡山駅 (OM08)** です。

そこから反対方向（自由が丘方面）の電車で一駅戻った駅は**緑が丘駅 (OM09)** です。

したがって、つばめちゃんの目的地の駅は**緑が丘駅**です。
```

## 45. マルチターン対話

先ほどの応答に続けて, 以下の追加の問いかけに対する応答を生成せよ.

> さらに、つばめちゃんが自由が丘駅で乗り換えたとき、先ほどとは反対方向の急行電車に間違って乗車してしまった場合を考えます。目的地の駅に向かうため、自由が丘の次の急行停車駅で降車した後、反対方向の各駅停車に乗車した場合、何駅先の駅で降りれば良いでしょうか？

```Python
import os
import httpx
import base64
import google.generativeai as genai

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

model = genai.GenerativeModel(
    model_name="gemini-2.5-pro-exp-03-25",
    generation_config={
        "temperature": 0.6,
        "top_p": 0.9,
    }
)

image_path = "https://www.tokyu.co.jp/railway/station/img/map_img01.jpg"
image = httpx.get(image_path)

system = "あなたは日本の鉄道員です。添付画像を参考にして、以下の問いに答えてください。\n"
user1 = "\n【問題】つばめちゃんは渋谷駅から東急東横線に乗り、自由が丘駅で乗り換えました。東急大井町線の大井町方面の電車に乗り換えたとき、各駅停車に乗車すべきところ、間違えて急行に乗車してしまったことに気付きました。自由が丘の次の急行停車駅で降車し、反対方向の電車で一駅戻った駅がつばめちゃんの目的地でした。目的地の駅の名前を答えてください。\n"

prompt = system + user1
response1 = model.generate_content([{'mime_type':'image/jpeg', 'data': base64.b64encode(image.content).decode('utf-8')}, prompt]).text

user2 = "\n【問題】\nさらに、つばめちゃんが自由が丘駅で乗り換えたとき、先ほどとは反対方向の急行電車に間違って乗車してしまった場合を考えます。目的地の駅に向かうため、自由が丘の次の急行停車駅で降車した後、反対方向の各駅停車に乗車した場合、何駅先の駅で降りれば良いでしょうか？\n"

prompt = prompt + response1 + user2
response2 = model.generate_content([{'mime_type':'image/jpeg', 'data': base64.b64encode(image.content).decode('utf-8')}, prompt]).text

print(f"System:\n\n{system}\nUser:\n{user1}\nGemini:\n\n{response1}\n\nUser:\n{user2}\nGemini:\n\n{response2}")
```

```bash
System:

あなたは日本の鉄道員です。添付画像を参考にして、以下の問いに答えてください。

User:

【問題】つばめちゃんは渋谷駅から東急東横線に乗り、自由が丘駅で乗り換えました。東急大井町線の大井町方面の電車に乗り換えたとき、各駅停車に乗車すべきところ、間違えて急行に乗車してしまったことに気付きました。自由が
丘の次の急行停車駅で降車し、反対方向の電車で一駅戻った駅がつばめちゃんの目的地でした。目的地の駅の名前を答えてください。

Gemini:

承知いたしました。鉄道員としてお答えします。

1.  つばめちゃんは渋谷駅から東急東横線で自由が丘駅（TY07 / OM10）まで来ました。
2.  自由が丘駅で東急大井町線（オレンジ色の路線）の大井町方面（駅番号が小さくなる方向）に乗り換えました。
3.  間違えて急行（オレンジ色の丸印が停車駅）に乗車しました。
4.  自由が丘駅（OM10）の次の急行停車駅は、大岡山駅（OM08）です。
5.  つばめちゃんは大岡山駅で降車しました。
6.  そこから反対方向（自由が丘方面、駅番号が大きくなる方向）の電車で一駅戻りました。
7.  大岡山駅（OM08）から自由が丘方面へ一駅戻った駅は、緑が丘駅（OM09）です。

したがって、つばめちゃんの目的地の駅は**緑が丘駅（みどりがおかえき）**です。

User:

【問題】
さらに、つばめちゃんが自由が丘駅で乗り換えたとき、先ほどとは反対方向の急行電車に間違って乗車してしまった場合を考えます。目的地の駅に向かうため、自由が丘の次の急行停車駅で降車した後、反対方向の各駅停車に乗車した
場合、何駅先の駅で降りれば良いでしょうか？

Gemini:

承知いたしました。鉄道員としてお答えします。

1.  つばめちゃんは自由が丘駅（OM10）で東急大井町線（オレンジ色の路線）に乗り換えました。
2.  本来は大井町方面（駅番号が小さくなる方向）へ行くべきところ、間違えて反対の溝の口方面（駅番号が大きくなる方向）の**急行**（オレンジ色の丸印が停車駅）に乗車しました。
3.  自由が丘駅（OM10）から溝の口方面への次の急行停車駅は、**二子玉川駅（OM15）**です。
4.  つばめちゃんは二子玉川駅（OM15）で降車しました。
5.  そこから反対方向（大井町方面、駅番号が小さくなる方向）の**各駅停車**に乗車しました。
6.  目的地の駅は、最初の問題で確認した通り**緑が丘駅（OM09）**です。
7.  二子玉川駅（OM15）から緑が丘駅（OM09）まで、各駅停車で何駅か数えます。
    *   二子玉川（OM15）出発
    *   1駅目：上野毛（OM14）
    *   2駅目：等々力（OM13）
    *   3駅目：九品仏（OM12）
    *   4駅目：尾山台（OM11）
    *   5駅目：自由が丘（OM10）
    *   6駅目：**緑が丘（OM09）** ← 目的地

したがって、二子玉川駅から各駅停車に乗車した場合、**6駅先**の駅で降りれば目的地の緑が丘駅に到着します。
```

## 46. 川柳の生成

適当なお題を設定し, 川柳の案を10個作成せよ.

```python
import os
import google.generativeai as genai

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

model = genai.GenerativeModel(
    model_name="gemini-2.0-flash",
    generation_config={
        "temperature": 0.6,
        "top_p": 0.9,
        "max_output_tokens": 512
    }
)

system = "あなたは日本の川柳家です\n"
user = "「春」をお題として川柳を10句作ってください。\n"

prompt = system + user

response = model.generate_content(prompt).text

print(response)
```

```bash
はい、承知いたしました。「春」をテーマに川柳を10句詠みます。

1.  蕾から　顔出す笑顔　春隣
2.  うぐいすの　下手な歌にも　春を知る
3.  ランドセル　背負う希望と　春の風
4.  菜の花や　黄色い絨毯　夢心地
5.  陽だまりに　猫もまどろむ　春の昼
6.  別れ際に　桜ひとひら　舞い降りる
7.  春霞　遠き山並み　ぼんやりと
8.  春眠や　過去も未来も　夢の中
9.  花見酒　酔えば上機嫌　皆笑顔
10. 春の雨　草木潤し　命芽吹く
```

## 47. LLMによる評価

大規模言語モデルを評価者 (ジャッジ) として, 問題46の川柳の面白さを10段階で評価せよ.

```python
import os
import google.generativeai as genai

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

model = genai.GenerativeModel(
    model_name="gemini-2.0-flash",
    generation_config={
        "temperature": 0.6,
        "top_p": 0.9,
        "max_output_tokens": 1024
    }
)

system = "あなたは日本の川柳評論家です。以下に、「春」をお題とした川柳を示します。それぞれの川柳の面白さを1～10の10段階で評価してください。\n"
user = """
1.  蕾から　顔出す笑顔　春隣
2.  うぐいすの　下手な歌にも　春を知る
3.  ランドセル　背負う希望と　春の風
4.  菜の花や　黄色い絨毯　夢心地
5.  陽だまりに　猫もまどろむ　春の昼
6.  別れ際に　桜ひとひら　舞い降りる
7.  春霞　遠き山並み　ぼんやりと
8.  春眠や　過去も未来も　夢の中
9.  花見酒　酔えば上機嫌　皆笑顔
10. 春の雨　草木潤し　命芽吹く
"""

prompt = system + user

response = model.generate_content(prompt).text

print(response)
```

```bash
はい、日本の川柳評論家として、春を題材にした川柳を評価させていただきます。それぞれの句の面白さを1から10の段階で評価し、簡単なコメントを添えます。

1.  **蕾から　顔出す笑顔　春隣 (7/10)**
    *   蕾が人の笑顔に見えるという擬人化が面白い。春の訪れを隣に感じるという表現も良い。ただし、やや説明的な印象も受ける。

2.  **うぐいすの　下手な歌にも　春を知る (8/10)**
    *   うぐいすの歌が上手下手で評価されるという視点がユニーク。完璧ではないものにも春を感じるという、余裕のある心持ちが伝わってくる。

3.  **ランドセル　背負う希望と　春の風 (9/10)**
    *   ランドセルと春の風という組み合わせが、新生活への希望を象徴的に表している。情景が目に浮かび、共感を呼ぶ。

4.  **菜の花や　黄色い絨毯　夢心地 (6/10)**
    *   菜の花畑を絨毯に見立てる表現は一般的だが、夢心地という言葉で、その美しさを強調している。やや平凡な印象。

5.  **陽だまりに　猫もまどろむ　春の昼 (8/10)**
    *   陽だまりと猫という、春の穏やかな情景を象徴する組み合わせが良い。のどかな雰囲気が伝わってくる。

6.  **別れ際に　桜ひとひら　舞い降りる (9/10)**
    *   別れという切ない場面に、桜の花びらが舞い降りる情景が美しい。出会いと別れの季節である春を象徴的に表している。

7.  **春霞　遠き山並み　ぼんやりと (7/10)**
    *   春霞という季語が、遠くの山並みをぼんやりとさせる情景をうまく描写している。静かで穏やかな春の雰囲気が伝わる。

8.  **春眠や　過去も未来も　夢の中 (6/10)**
    *   春眠という季語から、心地よい眠りの中で過去や未来に思いを馳せる様子が想像できる。ただし、やや抽象的な表現。

9.  **花見酒　酔えば上機嫌　皆笑顔 (8/10)**
    *   花見酒という春の風物詩と、酔って上機嫌になる人々の笑顔を描写している。明るく楽しい雰囲気が伝わる。

10. **春の雨　草木潤し　命芽吹く (7/10)**
    *   春の雨が草木を潤し、新しい命が芽吹く様子を描写している。春の生命力を感じる句。やや説明的な印象も受ける。

**総評：**

全体的に、春の情景や感情をうまく表現した川柳が多いと感じました。特に、ランドセル、別れ際の桜、猫のまどろみなど、具体的なイメージを喚起する句は、共感を呼びや
すく、印象に残ります。
```

## 48. LLMによる評価の頑健性

問題47で行ったLLMによるテキストの評価に関して, その頑健さ (脆弱さ) を調査せよ. 最も単純な方法は, 同じ評価を何回か繰り返した時のスコアの分散を調べることであろう. また, 川柳の末尾に特定のメッセージを追加することで, 評価スコアを恣意的に操作することも可能であろう.

```python
import os
import time
from tqdm import tqdm
import numpy as np
import google.generativeai as genai

interval = 4.1

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

model = genai.GenerativeModel(
    model_name="gemini-2.0-flash",
    generation_config={
        "temperature": 0.6,
        "top_p": 0.9
    }
)

N = 10
scores = []
for _ in tqdm(range(N), desc="Progress"):
    system = system = "あなたは日本の川柳評論家です。以下に、「春」をお題とした川柳を示します。それぞれの川柳の面白さを1～10の10段階で評価してください。\n"
    user1 = """
    1.  蕾から　顔出す笑顔　春隣
    2.  うぐいすの　下手な歌にも　春を知る
    3.  ランドセル　背負う希望と　春の風
    4.  菜の花や　黄色い絨毯　夢心地
    5.  陽だまりに　猫もまどろむ　春の昼
    6.  別れ際に　桜ひとひら　舞い降りる
    7.  春霞　遠き山並み　ぼんやりと
    8.  春眠や　過去も未来も　夢の中
    9.  花見酒　酔えば上機嫌　皆笑顔
    10. 春の雨　草木潤し　命芽吹く\n
    """

    prompt = system + user1
    response = model.generate_content(prompt).text
    time.sleep(interval)

    user2 = f"先ほどの回答\n「{response}」\nから評価値だけを抽出し、comma区切りでraw文字列として出力してください。ただし、コードブロックは不要です。\n"

    prompt = prompt + user2
    response = model.generate_content(prompt).text
    time.sleep(interval)

    arr = []
    arr = [int(x) for x in response.split(",")]
    scores.append(arr)

variance = np.var(scores)
print(f"Variance: {variance}")
```

```
Variance: 1.0275
```

## 49. トークン化

以下の文章 (夏目漱石の『吾輩は猫である』の冒頭部分) のトークン数を計測せよ.

> 吾輩は猫である。名前はまだ無い。
>
>どこで生れたかとんと見当がつかぬ。何でも薄暗いじめじめした所でニャーニャー泣いていた事だけは記憶している。吾輩はここで始めて人間というものを見た。しかもあとで聞くとそれは書生という人間中で一番獰悪な種族であったそうだ。この書生というのは時々我々を捕えて煮て食うという話である。しかしその当時は何という考もなかったから別段恐しいとも思わなかった。ただ彼の掌に載せられてスーと持ち上げられた時何だかフワフワした感じがあったばかりである。掌の上で少し落ちついて書生の顔を見たのがいわゆる人間というものの見始であろう。この時妙なものだと思った感じが今でも残っている。第一毛をもって装飾されべきはずの顔がつるつるしてまるで薬缶だ。その後猫にもだいぶ逢ったがこんな片輪には一度も出会わした事がない。のみならず顔の真中があまりに突起している。そうしてその穴の中から時々ぷうぷうと煙を吹く。どうも咽せぽくて実に弱った。これが人間の飲む煙草というものである事はようやくこの頃知った。

```python
import os
import google.generativeai as genai

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

model = genai.GenerativeModel(
    model_name="gemini-2.0-flash",
    generation_config={
        "temperature": 0.6,
        "top_p": 0.9,
        "max_output_tokens": 512
    }
)

system = "あなたは日本の文章校正者です。以下に示す文章のトークン数を計測してください。\n"
user = """
吾輩は猫である。名前はまだ無い。

どこで生れたかとんと見当がつかぬ。何でも薄暗いじめじめした所でニャーニャー泣いていた事だけは記憶している。吾輩はここで始めて人間というものを見た。しかもあとで聞くとそれは書生という人間中で一番獰悪な種族であったそうだ。この書生というのは時々我々を捕えて煮て食うという話である。しかしその当時は何という考もなかったから別段恐しいとも思わなかった。ただ彼の掌に載せられてスーと持ち上げられた時何だかフワフワした感じがあったばかりである。掌の上で少し落ちついて書生の顔を見たのがいわゆる人間というものの見始であろう。この時妙なものだと思った感じが今でも残っている。第一毛をもって装飾されべきはずの顔がつるつるしてまるで薬缶だ。その後猫にもだいぶ逢ったがこんな片輪には一度も出会わした事がない。のみならず顔の真中があまりに突起している。そうしてその穴の中から時々ぷうぷうと煙を吹く。どうも咽せぽくて実に弱った。これが人間の飲む煙草というものである事はようやくこの頃知った。
"""

prompt = system + user
response = model.generate_content(prompt).text

print(response)
```

```bash
この文章のトークン数は、MeCabなどの日本語形態素解析器を使うと正確に計測できますが、おおよその目安として以下のように数えることができます。

*   文の数: 12
*   句読点を含む総文字数: 611
*   空白を除いた文字数: 538

一般的に、日本語の文章では、文字数がおおよそのトークン数に近いと考えることができます。ただし、形態素解析器を使うと、より正確なトークン数を得られます。

**したがって、この文章のトークン数の目安は、約538トークンです。**
```